Run the cell below if you are using Google Colab to mount your Google Drive in your Colab instance. Adjust the path to the files in your Google Drive as needed if it differs.

If you do not use Google Colab, running the cell will simply do nothing, so do not worry about it.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd 'drive/My Drive/Colab Notebooks/04_Classification'
except ImportError as e:
    pass

## Exercise 4: Classification

### 4.1. Learning a classifier for the Iris Data Set – Part II

In the last exercise, you have learned classification models for the Iris dataset using a train/test split. Now try learning a decision tree and evaluate it with 10-fold cross-validation. Use a pipeline to perform some preprocessing before learning or applying the decision tree classifier. For this exercise, we use the ```iris_imbalanced.csv``` dataset, so it's a good idea to include a balancing step in the preprocessing!

#### 4.1.1.	Discretise the Iris data set into three bins. Then use the DecisionTreeClassifier with a 10-fold stratified cross validation and compute the accuracy. Afterwards plot the decision tree.

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

## Windows users: either add the path to graphviz' dot.exe to your PATH variable 
## OR comment in the 2 lines below (may have to change path):

# import os
# os.environ['PATH'] += ';C:\\Program Files (x86)\\Graphviz2.38\\bin'

# load the dataset
iris = pd.read_csv("iris_imbalanced.csv")
iris_data = iris[['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']]
iris_target = iris['Name']

# define a pipeline including a discretiser, balancing, and a decision tree classifier
#TODO: INSERT YOUR CODE HERE!

# evaluate the pipeline
accuracy = cross_val_score(pipeline, iris_data, iris_target, cv=10, scoring='accuracy')

# fit the pipeline to the dataset 
pipeline.fit(iris_data, iris_target)

print("Fitted a decision tree with {} nodes. Cross-validated accuracy is {}%".format(estimator.tree_.node_count, accuracy.mean() * 100))

# plot the decision tree
import graphviz 
from sklearn import tree

dot_data = tree.export_graphviz(estimator, out_file=None, 
                                feature_names=iris_data.columns, 
                                class_names=iris_target.unique(),
                               filled=True, rounded=True, special_characters=True) 

graph = graphviz.Source(dot_data) 
display(graph)

#### 4.1.2.	Remove the discretization and adjust the max_depth parameter of DecisionTreeClassifier to increase the accuracy. Does the accuracy change? Compare the complexity of the two models. Which model should be preferred according to Occam’s razor?

In [ ]:
# disable the discretiser
#TODO: INSERT YOUR CODE HERE!

# re-evaluate the pipeline
accuracy = cross_val_score(pipeline, iris_data, iris_target, cv=10, scoring='accuracy')

# fit the pipeline to the dataset again
pipeline.fit(iris_data, iris_target)

print("Fitted a decision tree with {} nodes. Cross-validated accuracy is {}%".format(estimator.tree_.node_count, accuracy.mean() * 100))

# plot the decision tree
dot_data = tree.export_graphviz(estimator, out_file=None, 
                                feature_names=iris_data.columns, 
                                class_names=iris_target.unique(),
                               filled=True, rounded=True, special_characters=True) 

graph = graphviz.Source(dot_data) 

display(graph)

In [ ]:
# adjust the max_depth parameter and re-evaluate
for depth in range(1, 9):
    #TODO: INSERT YOUR CODE HERE!
    
    accuracy = cross_val_score(pipeline, iris_data, iris_target, cv=10, scoring='accuracy')
    print("max_depth={}: {}% accuracy".format(depth, accuracy.mean()*100))

In [ ]:
# choose a good parameter setting and re-fit the pipeline to the dataset
#TODO: INSERT YOUR CODE HERE!


pipeline.fit(iris_data, iris_target)
dot_data = tree.export_graphviz(estimator, out_file=None, 
                                feature_names=iris_data.columns, 
                                class_names=iris_target.unique(),
                               filled=True, rounded=True, special_characters=True) 

graph = graphviz.Source(dot_data) 

display(graph)

Which model would you choose?

### 4.2. Who should get a bank credit?
The German credit data set from the UCI data set library (http://archive.ics.uci.edu/ml/index.html) describes the customers of a bank with respect to whether they should get a bank credit or not. The data set is provided as credit-g.arff file in ILIAS. 

#### 4.2.1. Plot ROC curves for k-NN (different k values) and Decision Tree (you can use the given avg_roc function) . Which classification approach looks most promising to you?

In [ ]:
from scipy.io import arff
credit_arff_data, credit_arff_meta = arff.loadarff(open('credit-g.arff', 'r'))
credit = pd.DataFrame(credit_arff_data)
# select all columns of type object
columns_with_binary_strings = credit.select_dtypes('object').columns.values

# decode the values of these columns using utf-8
credit[columns_with_binary_strings] = credit[columns_with_binary_strings].apply(lambda x: x.str.decode("utf-8"))
credit.head()

First define the preprocessing. Have a look at the class distribution and feature types and think about appropriate transformations.

In [ ]:
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt

# separate the target variable from the features
credit_target = credit['class']
credit_data = credit.drop('class', axis=1)

# plot the class distribution
#TODO: INSERT YOUR CODE HERE!


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# think about which features you want to re-scale, encode using one-hot encoding or encode using ordinal encoding
# then, create a ColumnTransformer to execute this preprocessing for you


Run the following cell once to define the ```avg_roc``` function:

In [ ]:
#define function for computing average roc for cross validation
#see http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html
from scipy import interp
from sklearn.metrics import roc_curve, auc

def avg_roc(cv, estimator, data, target, pos_label):
    mean_fpr = np.linspace(0, 1, 100) # = [0.0, 0.01, 0.02, 0.03, ... , 0.99, 1.0]
    tprs = []
    aucs = []
    
    for train_indices, test_indices in cv.split(data, target):
        train_data = data.iloc[train_indices]
        train_target = target[train_indices]
        estimator.fit(train_data, train_target)

        test_data = data.iloc[test_indices]
        test_target = target[test_indices]
        decision_for_each_class = estimator.predict_proba(test_data)#have to use predict_proba or decision_function 
    
        fpr, tpr, thresholds = roc_curve(test_target, decision_for_each_class[:,1], pos_label=pos_label)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0 # tprs[-1] access the last element
        aucs.append(auc(fpr, tpr))
        
        #plt.plot(fpr, tpr)# plot for each fold
        
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0 # set the last tpr to 1
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    
    return mean_fpr, mean_tpr, mean_auc, std_auc

The evaluate different classifiers using the ```avg_roc``` function and plot the results:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

# define the cross validation folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# create the pipeline, we will set the estimator later
pipeline = Pipeline([ ('preprocessing', preprocessor), ('estimator', None) ])

# setup a figure
plt.figure(figsize=(10,10))
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Random', alpha=.8) # draw diagonal

# KNN
#TODO: INSERT YOUR CODE HERE!
for n_neighbour in [2,3,4,5,6]:
    #TODO: INSERT YOUR CODE HERE!
    mean_fpr, mean_tpr, mean_auc, std_auc = avg_roc(cv, pipeline, credit_data, credit_target, 'good')
    plt.plot(mean_fpr, mean_tpr, label='{}-NN (AUC: {:.3f} $\pm$ {:.3f})'.format(n_neighbour, mean_auc, std_auc))
    
# Decision Tree
#TODO: INSERT YOUR CODE HERE!
mean_fpr, mean_tpr, mean_auc, std_auc = avg_roc(cv, pipeline, credit_data, credit_target, 'good')
plt.plot(mean_fpr, mean_tpr, label='DecisonTree (AUC: {:.3f} $\pm$ {:.3f})'.format(mean_auc, std_auc))

# show the plot
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.legend()
plt.show()

#### 4.2.2.	For the two most promising classification approaches, compute the accuracy and confusion matrix in a 10-fold CV setup. Which level of accuracy do you reach?

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Plot confusion matrix and classification report for the two most promising approaches


#### 4.2.3.	What do the precision and recall values for the class “bad”  tell you? Try to improve the situation by increasing the number of “bad” examples in the training set (in the cross validation). How do precision and recall change if you apply this procedure? 

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# upsample the "bad" examples and evaluate


#### 4.2.4.	To model a use case specific evaluation, compute the cost of all missclassifications. Set up your cost matrix by assuming that you will lose 1 Unit if you refuse a credit to a good customer, but that you lose 100 Units if you give a bad customer a credit. Rerun the experiments and evaluate the results.

In [ ]:
from sklearn.metrics import confusion_matrix

# create the predictions here
# prediction = ...

# You can use the following code to calculate the cost
cm = confusion_matrix(credit_target, prediction, labels=credit_target.unique())
cost = cm[0][1] * 100 + cm[1][0] * 1
acc = accuracy_score(credit_target, prediction)
print("6-NN with accuracy of {} and cost {}".format(acc, cost))

#### 4.2.5.	As the creation of training data is mostly a manual task and humans tend to be fallible, training data might include noise. Simulate this behavior by using the Add Noise function and change the parameter “percentage” from 0% over 10% to 20%. Is your preferred classification approach still feasible for this situation? How does the performance of the other classifiers evolve?

In [ ]:
import random 
from sklearn.utils.multiclass import unique_labels
def add_noise(raw_target, percentage):    
    labels = unique_labels(raw_target)
    target_with_noise = []
    for one_target_label in raw_target:
        if random.randint(1,100) <= percentage:
            target_with_noise.append(next(l for l in labels if l != one_target_label))
        else:
            target_with_noise.append(one_target_label)
    return target_with_noise

In [ ]:
# TODO: add noise to your training splits and evaluate (e.g. use manual cross validation (see intro slides) for this)